EXTRACT

In [1]:
import pandas as pd

# EXTRACT
df = pd.read_csv("transactions_ip.csv")
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')

# Simpan hasil ekstraksi
df.to_csv("/tmp/extracted_payment.csv", index=False)
df.head()

,transaction_id,customer_id,transaction_date,total_amount,payment_method,shipping_method,delivery_time,coupon_code,city,product_id,sales_factor
0,1,506,2023-07-22,11300000.0,Cash on Delivery,Tiki,1 minggu,GRATISONGKIR,Jakarta,12,0.7
1,2,23275,2024-07-01,11053000.0,DANA,Tiki,2-3 hari,SALE20,Bandung,3,0.9
2,3,24101,2022-11-01,12741000.0,Transfer Bank,JNE,4-5 hari,NaN,Bandung,5,0.8
3,4,24188,2022-02-04,4921000.0,OVO,JNE,4-5 hari,SALE20,Yogyakarta,10,0.6
4,5,33766,2022-02-06,2163000.0,Transfer Bank,GrabExpress,1 minggu,NaN,Jakarta,11,0.7


TRANSFORM

dim_date

In [2]:
dim_date = df[['transaction_date']].drop_duplicates().copy()
dim_date['date_id'] = dim_date['transaction_date'].dt.strftime('%Y%m%d').astype(int)
dim_date['year'] = dim_date['transaction_date'].dt.year
dim_date['month'] = dim_date['transaction_date'].dt.month
dim_date['day'] = dim_date['transaction_date'].dt.day

dim_customer

In [3]:
dim_customer = df[['customer_id']].drop_duplicates().copy()

dim_payment_method

In [4]:
dim_payment_method = df[['payment_method']].drop_duplicates().copy()
dim_payment_method['payment_method_id'] = dim_payment_method.index + 1

dim_coupon

In [5]:
df['coupon_code'] = df['coupon_code'].fillna('NO_COUPON')

dim_coupon = df[['coupon_code']].drop_duplicates().copy()
dim_coupon['coupon_id'] = dim_coupon.index + 1

fact_payment

In [6]:
fact_payment = df.merge(dim_date, on='transaction_date') \
                 .merge(dim_payment_method, on='payment_method') \
                 .merge(dim_coupon, on='coupon_code')

fact_payment = fact_payment[[
    'transaction_id', 'customer_id', 'date_id',
    'payment_method_id', 'coupon_id',
    'total_amount', 'sales_factor'
]]

LOAD

In [7]:
dim_date.to_csv("/tmp/dim_date_payment.csv", index=False)
dim_customer.to_csv("/tmp/dim_customer_payment.csv", index=False)
dim_payment_method.to_csv("/tmp/dim_payment_method.csv", index=False)
dim_coupon.to_csv("/tmp/dim_coupon.csv", index=False)
fact_payment.to_csv("/tmp/fact_payment.csv", index=False)
